In [212]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
import cv2
import h5py

In [213]:
dict_im = pickle.load( open( "/media/DATA/jbonato/astro_segm/notebook/inference_ex.pkl", "rb" ))
folder_save = '/media/DATA/jbonato/astro_segm/Aqua_binding/Contours/SMALL_'

In [214]:
num_list = ['2']


In [215]:
for num in num_list:
    query = '{0:0>3}'.format(num)
    num_cell = dict_im['Cell_num_'+query]
    map_cells = np.zeros((num_cell,256,256))

    for k in range(num_cell):
        cell_query= '{0:0>3}'.format(str(k))
        for key in dict_im['ROI_'+query]:
            if ('Soma_'+cell_query in key ) or ('Proc_'+cell_query in key ) or ('CC_'+cell_query in key):
                c1,c2 = dict_im['ROI_'+query][key]
                map_cells[k,c1,c2]+=1

    dilate_ROI = 2
    BB = np.zeros_like(map_cells)
    for k in range(num_cell):#
        buff = map_cells[k].astype(np.uint8)
        for _ in range(dilate_ROI):
            element = cv2.getStructuringElement(cv2.MORPH_CROSS, (3,3))
            buff = cv2.dilate(buff, element)

        buff[buff>=1]=255
        _,thresh = cv2.threshold(buff,1,255,0)
        mask = np.zeros((256+2, 256+2), np.uint8)

        cv2.floodFill(thresh, mask, (0,0), 255);
        mask = cv2.bitwise_not(mask)
        mask[mask!=255]=0
        mask = mask[1:-1,1:-1]
        # find contours in the binary image
        contours, _= cv2.findContours(mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
        max_ =0  
        for cnt in contours:
            print(cnt.shape[0],max_)
            if cnt.shape[0]>=max_:
                cont_max = cnt
                max_ = cnt.shape[0]

        box =  cv2.boundingRect(cont_max)        
        BB[k,box[1]:box[1]+box[3],box[0]:box[0]+box[2]]=1

        mask[mask==255]=1
        map_cells[k] = mask.astype(np.uint8)

        with h5py.File(folder_save +query+'.hdf5','w') as f:
            dset = f.create_dataset('cc',data=map_cells)
            dset2 = f.create_dataset('bb',data=BB)


20 0
518 20
392 0
307 0
56 0
125 56
44 125
118 0
